In [ ]:
%matplotlib inline
## imports

# site
from scipy.ndimage.morphology import binary_erosion
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import rasterio
import flopy

# std
from pathlib import Path
import yaml

In [ ]:
def read_array(rasterfile, masked=True,  band=1):
    with rasterio.open(rasterfile) as src:
        return src.read(band, masked=masked)

In [ ]:
def read_profile(rasterfile):
    with rasterio.open(rasterfile) as src:
        return src.profile

In [ ]:
def write_array(rasterfile, values, profile):
    with rasterio.open(rasterfile, 'w', **profile) as dst:
        return dst.write(values, 1)

In [ ]:
def read_3d_array(rasterfiles, stack_axis=0, masked=True):
    arrays = []
    for rasterfile in rasterfiles:
        arrays.append(read_array(rasterfile, masked=masked))
    return np.ma.stack(arrays, axis=stack_axis)

In [ ]:
## input
# name
name = 'test_run'

# workspace
workspace = Path(r'..\output\test_run')

# exe name
exe_name = Path(r'..\bin\mf6.0.3\bin\mf6.exe')

# spatial reference
xllcorner = 60_000.
yllcorner = 322_500.

# grid dimensions
nlay = 19
nrow = 450
ncol = 601
delr = 250.
delc = 250.

# data files
topfile = Path(r'..\data\topbot\RL{ilay:d}.tif')
botfile = Path(r'..\data\topbot\TH{ilay:d}.tif')
idomainfile = Path(r'..\data\boundary\ibound.tif')
               
kdfile = Path(r'..\data\kdc\TX{ilay:d}.tif')
cfile = Path(r'..\data\kdc\CL{ilay:d}.tif')
             
startfile = Path(r'..\data\startingheads\HH{ilay:d}.tif')

rechargefile = Path(r'..\data\recharge\RP1.tif')

drn2005file = Path(r'..\data\mf2005\drn_data.csv')
ghb2005file = Path(r'..\data\mf2005\ghb_data.csv')
riv2005file = Path(r'..\data\mf2005\riv_data.csv')

sqfile = Path(r'..\data\wells\sq_list.csv')

In [ ]:
# create workspace directory
workspace.mkdir(exist_ok=True)

In [ ]:
# Create the Flopy simulation object
sim = flopy.mf6.MFSimulation(
    sim_name=name,
    exe_name=str(exe_name), 
    version='mf6',
    sim_ws=str(workspace))

# Create the Flopy temporal discretization object
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim,
    pname='tdis',
    time_units='DAYS',
    nper=1, 
    perioddata=[(1.0, 1, 1.0)],
    )

# Create the Flopy groundwater flow (gwf) model object
model_nam_file = '{}.nam'.format(name)
gwf = flopy.mf6.ModflowGwf(sim,
    modelname=name, 
   model_nam_file=model_nam_file,
   )

In [ ]:
# read tops
topfiles = (topfile.parent / topfile.name.format(ilay=i + 1) for i in range(nlay))

tops = read_3d_array(topfiles)

# mask bad nodata values
tops = np.ma.masked_where(tops.mask | (tops < -9990.), tops)

In [ ]:
# read bots
botfiles = (botfile.parent / botfile.name.format(ilay=i + 1) for i in range(nlay))

bots = read_3d_array(botfiles)

# mask bad nodata values
bots = np.ma.masked_where(bots.mask | (bots < -9990.), bots)

In [ ]:
# convert to top, botm
top = tops[0, :, :].filled(0.)
botm = []
for ilay in range(nlay):
    botm.append(bots[ilay, :, :].filled((2*ilay + 1) * -1e-3))
    if (ilay + 1) < nlay:
        botm.append(tops[ilay + 1, :, :].filled((2*ilay + 2) * -1e-3))
botm = np.ma.stack(botm)

In [ ]:
# read idomain
idomain = read_array(idomainfile).filled(0.)

In [ ]:
# idomain
idomain3d = np.broadcast_to(idomain, botm.shape)

In [ ]:
# initialize the DIS package
dis = flopy.mf6.modflow.mfgwfdis.ModflowGwfdis(gwf,
    pname='dis', nlay=(nlay*2 - 1),
    nrow=nrow, ncol=ncol,
    delr=delr, delc=delc,
    top=top, botm=botm,
    idomain=idomain3d,
    )

In [ ]:
# read kD
kdfiles = (kdfile.parent / kdfile.name.format(ilay=i + 1) for i in range(nlay))
kd = read_3d_array(kdfiles)

# convert to kh
kh = (kd / (tops - bots))

# fill with low value
kh = kh.filled(1e-6)

In [ ]:
# add dummy values
kh3d = np.ones((nlay*2 - 1, nrow, ncol)) * 1e-6
kh3d[0::2] = kh

In [ ]:
# read c
cfiles = (cfile.parent / cfile.name.format(ilay=i + 1) for i in range(nlay - 1))
c = read_3d_array(cfiles)

# convert to kv
kv = (bots[:-1, :, :] - tops[1:, :, :]) / c

# fill with high value
kv = kv.filled(1e6)

In [ ]:
# add dummy values
kv3d = np.ones((nlay*2 - 1, nrow, ncol)) * 1e6
kv3d[1::2] = kv

In [ ]:
# initialize the NPF package
npf = flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf(
    model=gwf,
    k=kh3d,
    k22=kh3d,
    k33=kv3d,
)

In [ ]:
# read start
startfiles = (startfile.parent / startfile.name.format(ilay=i + 1) for i in range(nlay))
start = np.ma.repeat(read_3d_array(startfiles), 2, axis=0)[:-1, :, :]

# mask values larger than 1000
start = np.ma.masked_where(start.mask | (start > 1e3), start)

# fill masked with zeros
start = start.filled(0.)

In [ ]:
# Create the initial conditions package
ic = flopy.mf6.modflow.mfgwfic.ModflowGwfic(gwf, pname='ic', strt=start)

In [ ]:
# get boundary data from idomain and starting heads
structure = np.ones((3, 3))
isboundary = (idomain == 1) & ~binary_erosion(idomain, structure)
chd_data = []
for ilay in range(dis.nlay.get_data()):
    for row, col in np.ndindex(*isboundary.shape):
        if isboundary[row, col]:
            chd_data.append(((ilay, row, col), start[ilay, row, col]))

In [ ]:
# initialize the CHD package
chd = flopy.mf6.modflow.mfgwfchd.ModflowGwfchd(gwf, pname='chd', stress_period_data=chd_data)

In [ ]:
# recharge
recharge = read_array(rechargefile)
rch_rec = []
lay = 0
for row, col in np.ndindex(*recharge.shape):
    if idomain[row, col]:
        rch_rec.append(((lay, row, col), recharge[row, col]))
rch_data = {0: rch_rec}

In [ ]:
# initialize the RCH package
rch = flopy.mf6.ModflowGwfrch(gwf, 
   pname='rch', stress_period_data=rch_data)

In [ ]:
# DRN stress period data from Modflow 2005
drn2005_data = pd.read_csv(drn2005file)

# select active cells
is_active = (idomain == 1)[drn2005_data.loc[:, 'i'], drn2005_data.loc[:, 'j']]

get_stress_record = lambda r: (
    (int(r.loc['k']) * 2, int(r.loc['i']), int(r.loc['j'])), r.loc['elev0'], r.loc['cond0']
    )
drn_data = drn2005_data.iloc[is_active, :].apply(get_stress_record, axis=1).values.tolist()

# initialize the DRN package
drn = flopy.mf6.modflow.mfgwfdrn.ModflowGwfdrn(gwf, pname='drn', stress_period_data=drn_data)

In [ ]:
# GHB stress period data from Modflow 2005
ghb2005_data = pd.read_csv(ghb2005file)

# select active cells
is_active = (idomain == 1)[ghb2005_data.loc[:, 'i'], ghb2005_data.loc[:, 'j']]

get_stress_record = lambda r: (
    (int(r.loc['k']) * 2, int(r.loc['i']), int(r.loc['j'])), r.loc['bhead0'], r.loc['cond0']
    )
ghb_data = ghb2005_data.iloc[is_active, :].apply(get_stress_record, axis=1).values.tolist()

# initialize the GHB package
ghb = flopy.mf6.modflow.mfgwfghb.ModflowGwfghb(gwf, pname='ghb', stress_period_data=ghb_data)

In [ ]:
# RIV stress period data from Modflow 2005
riv2005_data = pd.read_csv(riv2005file)

# select active cells
is_active = (idomain == 1)[riv2005_data.loc[:, 'i'], riv2005_data.loc[:, 'j']]

get_stress_record = lambda r: (
    (int(r.loc['k']) * 2, int(r.loc['i']), int(r.loc['j'])), r.loc['stage0'], r.loc['cond0'], r.loc['rbot0']
    )
riv_data = riv2005_data.iloc[is_active, :].apply(get_stress_record, axis=1).values.tolist()

# initialize the RIV package
riv = flopy.mf6.modflow.mfgwfriv.ModflowGwfriv(gwf, pname='riv', stress_period_data=riv_data)

In [ ]:
# read sources data from csv file
sqs = pd.read_csv(sqfile)

# to row,col from x,y
fwd = rasterio.transform.from_origin(xllcorner, yllcorner + nrow*delr, delc, delr)

# transform xy to row,col
rows, cols = rasterio.transform.rowcol(fwd, sqs['xcoordinate'], sqs['ycoordinate'])

# layer numbers & pumping rates
layernumbers = sqs.loc[:, 'ilay'].tolist()
pumping_rates = sqs.loc[:, 'q_assigned'].tolist()

In [ ]:
# convert source data to stress period data records
wel_data = {}
wel_rec = []
for lay, row, col, q in zip(layernumbers, rows, cols, pumping_rates):
    if idomain[row, col]:
        wel_rec.append((((lay - 1)*2, row, col), q))
wel_data[0] = wel_rec

In [ ]:
# initialize WEL package
wel = flopy.mf6.modflow.mfgwfwel.ModflowGwfwel(gwf,
    pname='wel', 
    stress_period_data=wel_data)

In [ ]:
# Create the Flopy iterative model solver (ims) Package object
ims = flopy.mf6.modflow.mfims.ModflowIms(sim, pname='ims', complexity='SIMPLE')

In [ ]:
# Create the output control package
headfile = '{}.hds'.format(name)
head_filerecord = [headfile]
budgetfile = '{}.cbb'.format(name)
budget_filerecord = [budgetfile]
saverecord = [('HEAD', 'ALL'), 
              ('BUDGET', 'ALL')]
printrecord = [('HEAD', 'LAST')]
oc = flopy.mf6.modflow.mfgwfoc.ModflowGwfoc(gwf, pname='oc', saverecord=saverecord, 
                                            head_filerecord=head_filerecord,
                                            budget_filerecord=budget_filerecord,
                                            printrecord=printrecord)

In [ ]:
# write simulation to new location
sim.write_simulation()

In [ ]:
# Run the simulation
success, buff = sim.run_simulation()
print('\nSuccess is: ', success)

In [ ]:
# read heads
headfile = workspace / '{name:}.hds'.format(name=name)
hds = flopy.utils.binaryfile.HeadFile(headfile)

In [ ]:
# export to raster
rasterfolder = workspace / 'heads'
rasterfolder.mkdir(exist_ok=True)

profile = read_profile(idomainfile)

heads = hds.get_data()
for i, raster in enumerate(heads[::2]):  # export only horizontal flow layers i.e. 1, 3, .. 37
    raster[idomain < 1] = profile['nodata']  
    rasterfile = rasterfolder / 'phi{ilay:d}.tif'.format(ilay=i + 1)
    write_array(rasterfile, raster, profile)